In [11]:
import pydap
import netCDF4 as nc
import pandas as pd
import h5py
import xarray as xr
import geopy.distance
import numpy as np
import os

## Geographic distance between two points in km

In [2]:
def getDist(c1,c2):
    return geopy.distance.geodesic(c1, c2).km

In [15]:
gosat_path = './gosat/2016/csv_folder/'
oco2_path = './data_2016_oco2/csv_folder/'

In [24]:
# combining datasets
oco1 = pd.read_csv(os.path.join(oco2_path,'oco2_1ndjan2016.csv'))
oco2 = pd.read_csv(os.path.join(oco2_path,'oco2_2ndjan2016.csv'))
oco3 = pd.read_csv(os.path.join(oco2_path,'oco2_3rdjan2016.csv'))

In [25]:
oco1.append(oco2).append(oco3)

/var/folders/wx/g95tcl_55cg_v5h5q3hkngmh0000gn/T/ipykernel_2034/103467373.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  oco1.append(oco2).append(oco3)


,Unnamed: 0,latitude,longitude,time,sensor_zenith_angle,xco2,xco2_uncertainty,xco2_apriori,dates
0,0,-3.628539,-157.43341,1.451606e+09,22.461530,400.44990,0.393150,399.77917,2016-01-01 00:00:00.000
1,1,-3.639151,-157.43082,1.451606e+09,22.516706,400.22430,0.386413,399.77840,2016-01-01 00:00:00.000
2,2,-3.649849,-157.42830,1.451606e+09,22.573343,400.45795,0.384631,399.77770,2016-01-01 00:00:00.000
3,3,-3.660598,-157.42587,1.451606e+09,22.631292,399.87880,0.381401,399.78036,2016-01-01 00:00:00.000
4,4,-3.671286,-157.42354,1.451606e+09,22.689938,400.19666,0.379429,399.79030,2016-01-01 00:00:00.000
...,...,...,...,...,...,...,...,...,...
184035,184035,35.151627,-150.97835,1.451862e+09,46.409540,403.67450,0.487216,405.13090,2016-01-03 23:03:51.000
184036,184036,35.138103,-150.96680,1.451862e+09,46.476120,403.66240,0.472273,405.13605,2016-01-03 23:03:51.000
184037,184037,35.169296,-150.98486,1.451862e+09,46.420950,404.34457,0.474153,405.12418,2016-01-03 23:03:51.300
184038,184038,35.155770,-150.97331,1.451862e+09,46.487540,403.93440,0.516165,405.12933,2016-01-03 23:03:51.300


### Get points that are close to each other

In [16]:
combined_data = pd.DataFrame(columns = ["dates","latitude","longitude","time",'sensor_zenith_angle', 'xco2', 'xco2_uncertainty', 'xco2_apriori' ])
THRESHOLD = 3
for gosat in os.listdir(gosat_path):    
    df_gosat = pd.read_csv(os.path.join(gosat_path,gosat))
    gosat_index = 0
    for oco2 in os.listdir(oco2_path):
        df_oco2 = pd.read_csv(os.path.join(oco2_path,oco2))
        oco2_index = 0
        dist = []
        while oco2_index < len(df_oco2):
            oco2_latitude = df_oco2.iloc[oco2_index]["latitude"]
            oco2_longitude = df_oco2.iloc[oco2_index]["longitude"]
            oco2_time = df_oco2.iloc[oco2_index]["dates"]
            oco2_point = (oco2_latitude,oco2_longitude)
            while gosat_index < len(df_gosat):
                gosat_latitude = df_gosat.iloc[gosat_index]["latitude"]
                gosat_longitude = df_gosat.iloc[gosat_index]["longitude"]
                gosat_point = (gosat_latitude,gosat_longitude)
                dist.append(getDist(oco2_point,gosat_point))
                if dist<THRESHOLD:
                    print('Found close points')
                    combined_xco2 = (df_oco2.iloc[oco2_index]["xco2"] + df_gosat.iloc[gosat_index]["xco2"]) / 2
                    combined_data = combined_data.append({"time": oco2_time, "latitude": oco2_latitude, "longitude": oco2_longitude, "xco2": combined_xco2},ignore_index=True)
                gosat_index += 1
            oco2_index += 1
    

# Save the combined data to a CSV file
combined_data.to_csv("combined_data.csv", index=False)


In [17]:
combined_data

,dates,latitude,longitude,time,sensor_zenith_angle,xco2,xco2_uncertainty,xco2_apriori


In [10]:
max(dist)

16677.370627958942